In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model

In [3]:
labels_dict={0:'Mask',1:'No Mask'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [4]:
new = load_model('2.h5')

In [5]:
def predict(image):
    image = img_to_array(image)
    image = preprocess_input(image)
    image = image.reshape(-1, 224, 224, 3)
    return new.predict(image)

In [6]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    faces = face_cascade.detectMultiScale(img)
    for (x,y,w,h) in faces:
        
        face = img[y:y+h, x:x+w]
        face_re = cv2.resize(face,(224,224))
        pred = predict(face_re)
        
        label = np.argmax(pred,axis=1)[0]
        acc = np.max(pred,axis=1)[0]*100
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label] + ' ' + str(np.floor(acc)), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    cv2.imshow("FACE MASK DETECTOR", img)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cv2.destroyAllWindows()
cap.release()